# 📊 Modelo de Datos - E-Commerce

## 🧱 Tabla de Hechos Principal: `detalle_orden`

La tabla `detalle_orden` representa el **evento clave** que genera los datos del sistema: la compra de productos. Cada fila de esta tabla indica una **línea individual de una orden**, incluyendo:

- Producto
- Cantidad
- Precio unitario
- Referencia a la orden y al usuario

Esta tabla es el centro del modelo de datos por las siguientes razones:

- ✅ Alta **granularidad**: permite análisis por producto, usuario y fecha.
- ✅ Posee hechos cuantificables (cantidad, precio, total).
- ✅ Actúa como el **registro transaccional principal**, clave para medir el rendimiento del negocio.
- ✅ Es donde se genera el rédito económico del sistema.

> El E-Commerce necesita analizar órdenes por producto, fecha y usuario. Modelar correctamente esta tabla permite optimizar ventas, mejorar disponibilidad de productos y comprender mejor el comportamiento de los usuarios.

### 🔑 Clave de negocio

La clave compuesta `OrdenID + ProductoID` permite vincular múltiples dimensiones:  
📅 Fecha, 👤 Usuario, 💳 Método de pago, 📦 Producto, etc.

---

## 🛰️ Tablas de Hechos Secundarias

Otras tablas como `ordenes` pueden verse como hechos secundarios o bien como dimensiones si cumplen un rol más descriptivo (fecha, estado, total, usuario).

---

## 📐 Tipo de Modelo: Transaccional

Este modelo está optimizado para **registrar eventos en tiempo real**, priorizando:

- Normalización (evita redundancias y mejora la consistencia de los datos)
- Integridad referencial
- Capacidad de registrar grandes volúmenes de transacciones

---

## ❄️ Modelo de Datos: Snowflake

Se utiliza un esquema **Snowflake** debido a la presencia de:

- Una **tabla de hechos central** (`detalle_orden`)
- Múltiples dimensiones **conectadas jerárquicamente**
- Mini-dimensiones que brindan contexto adicional (por ejemplo: usuario, categoría de producto, fecha)

Este enfoque permite **mayor precisión, control relacional y escalabilidad**.

---

## 🖼️ Diagrama Conceptual

![Diagrama E-R Conceptual](downloads/images/diagrama_conceptual.png)

> Diagrama con las entidades principales, relaciones y reglas generales del negocio.  
Se observa cómo **todo parte desde la tabla `detalle_orden`**.

---

## 🧱 Diagrama Físico (Resumen)

**Nuevas columnas y optimizaciones propuestas:**

- 🕓 `fecha_registro` en la tabla `usuarios`
- 🔍 Índices para las tablas:
  - `detalle_orden` (por `producto_id`, `orden_id`)
  - `productos`
  - `usuarios`
- 📅 Partición por fecha en la tabla `ordenes`, para acelerar métricas temporales como evolución de ventas mensuales.
- Crear una nueva tabla intermedia llamada `orden_direccion_envio`, con una relación **1:1** con la tabla `orden`, para almacenar una **copia exacta de la dirección utilizada al momento de realizar el pedido**. Esta tabla garantizará que las direcciones asociadas a cada orden **no se vean afectadas por cambios posteriores en el perfil del usuario**.
🖼️ *La tabla `orden_direccion_envio` actúa como una snapshot de la dirección de envío al momento de la transacción.*

---

---

## ♻️ Slowly Changing Dimensions (SCD)

### 📌 Propuesta:

- 📦 En la tabla `ordenes`, actualmente solo se muestra el **último estado** de la orden (`Pendiente`, `Enviado`, `Cancelado`, etc.).  
  Se propone aplicar un **SCD Tipo 6** para almacenar el historial de cambios de estado. Agregando las columnas created_at, updated_at para ver la evolucion de la orden a lo largo del tiempo. Con este dato podriamos sacar insights sobre el ciclo de vida de una orden. 


---

## 📊 Tablas de Hechos Agregadas y Snapshots

---

### 🧮 Tablas de Hechos Agregadas

Para reducir la granularidad de `resena_productos`, se propone una tabla agregada por producto que incluya:

- ⭐ Promedio de calificación
- 🧾 Total de reseñas
- 📆 Agrupación por fecha (ej. por mes)

> Útil para mostrar calificaciones públicas en la página del producto, rankings, y analizar evolución temporal.

---

### 📸 Snapshots de Estado

Se recomienda generar una **snapshot semanal** de órdenes en estado "Pendiente":

```text
snapshot_ordenes_pendientes_2025_07_24
```